In [ ]:
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient
import os
import json

# Configurações do Azure Data Lake
account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
sas_token = os.getenv('AZURE_STORAGE_SAS_TOKEN')
account_url = f'https://{account_name}.blob.core.windows.net'

# Configurações do MongoDB
mongodb_uri = os.getenv('MONGODB_CONNECTION_STRING')
mongodb_database = os.getenv('MONGODB_DATABASE')

# Conexão ao MongoDB para listar coleções
client = MongoClient(mongodb_uri)
db = client[mongodb_database]
collections = db.list_collection_names()

blob_service_client = BlobServiceClient(account_url=f'{account_url}?{sas_token}')

# Caminho do contêiner
landing_container = f'abfs://landing-zone@{account_name}.dfs.core.windows.net'

# Processar as coleções do MongoDB
for collection_name in collections:
    print(f'Lendo coleção: {collection_name}')
    try:
        collection = db[collection_name]
        content = collection.find()
        documents = list(content)

        if not documents:
            print(f'A coleção {collection_name} está vazia.')
        else:
            for record in documents:
                for key in record:
                    record[key] = str(record[key])
        
            json_data = json.dumps(documents, indent=4)

            blob_name = f'{collection_name}.json'
            blob_client = blob_service_client.get_blob_client(container='landing-zone', blob=blob_name)
            blob_client.upload_blob(json_data, overwrite=True)

            print(f'Dados da coleção '{collection_name}' enviados para {blob_name}')
    except Exception as e:
        print(f'Erro ao processar a coleção {collection_name}: {e}')



Lendo coleção: cars
Dados da coleção 'cars' enviados para cars.json
Lendo coleção: sales
Dados da coleção 'sales' enviados para sales.json
Lendo coleção: services
Dados da coleção 'services' enviados para services.json
Lendo coleção: employees
Dados da coleção 'employees' enviados para employees.json
Lendo coleção: payments
Dados da coleção 'payments' enviados para payments.json
Lendo coleção: clients
Dados da coleção 'clients' enviados para clients.json
